In [37]:
# All unaccounted for variables in this file should be drawn from here (will put this in gitignore)
from wealth_modelling_variables import *

# Classes to help track overall wealth
from bank_account import *
from mortgage import *
from student_loan import *

# Functions to help model cashflow
from cost_modelling import *
from salary_modelling import *
from tax_modelling import *

# Oh and of course
import numpy as np
import matplotlib.pyplot as plt

In [38]:
# Initialising our 'account' and 'loan' objects
CurrentAccount = BankAccount(curent_account_balance, current_account_interest)
# In this version we go and get ourselves a Mortgage straight away!
# Future work should address the question of how this model looks if we
# get one at a later stage.
Mortgage = Mortgage(CurrentAccount, mortgage_deposit, initial_mortgage_value, repayment_term, monthly_payments, fixed_rate, fixed_rate_period)
StudentLoan = StudentLoan(CurrentAccount, initial_student_loan_value, student_loan_bands, student_loan_rates)
Pension = BankAccount(current_pension_value, pension_interest_rate)

In [30]:
wealth_increments = []

costs = model_costs(years, monthly_rent)

salaries = model_salary(years, starting_salary, annual_salary_increase, bonus_percentage) 
for i,salary in enumerate(salaries):

    # First, consider the effect on our student loan
    student_loan_repayment = StudentLoan.make_minimum_repayment(salary)
    # A curious instance of loan whose interest rate is contingent on my salary...
    StudentLoan.compound_interest(salary)

    # Next, address our pension contribution
    Pension.pay_in(salary*(pension_contribution + employer_pension_contribution))
    Pension.compound_interest()
    salary *= 1 - pension_contribution

    # Now apply income tax and NI
    tax = get_tax(salary, ni_bands, ni_rates, it_bands, it_rates)

    # Finally, take out our mortgage payment
    mortgage_repayment = Mortgage.make_yearly_repayment()
    Mortgage.compound_interest()

    # Oh and also get income from tenants in property
    tenant_income = tenant_monthly_rent * 12 * tenant_no

    # Now throw it all together and pay it into our Current Account
    income = salary - tax - costs[i] - student_loan_repayment - mortgage_repayment + tenant_income
    CurrentAccount.pay_in(income)
    CurrentAccount.compound_interest()

    wealth_increments += [CurrentAccount.value + Pension.value + Mortgage.AssetValue - Mortgage.loan_value - StudentLoan.loan_value]

In [ ]:
plt.figure(figsize=(15,10))

# Plot wealth trajectories.
plt.plot(np.arange(0,years,1)+2022, wealth_increments, label='SL-NR; M-22')
#plt.plot(np.arange(0,years,1)+2022,PhD_cumulative, label='PhD')
#plt.plot(np.arange(0,years,1)+2022,year_delay_PhD_cumulative, label='PhD +1 year deferral')

# Formatting the plot.
plt.xlabel('Years from FY 2022/23',fontsize=20)
plt.ylabel('Total Accumulated Wealth',fontsize=20)
plt.tick_params(labelsize=15)
l = plt.legend(loc=0,prop={'size': 15},title='Wealth Trajectories') 
plt.setp(l.get_title(),fontsize=15)
plt.show()